<a href="https://colab.research.google.com/github/nahubn1/Hybrid-Robot-Navigation-System/blob/main/notebooks/model_prototyping/U-Net-FiLM-Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import os
COLAB_PROJECT_ROOT = '/content/Thesis_Project'
if not os.path.exists(COLAB_PROJECT_ROOT):
  !git clone [your-repo-url] {COLAB_PROJECT_ROOT}
os.chdir(COLAB_PROJECT_ROOT) # Change directory into the project
!git pull # Ensure it's the latest version

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 1.34 KiB | 688.00 KiB/s, done.
From https://github.com/nahubn1/Hybrid-Robot-Navigation-System
   2423c22..a0758bf  main       -> origin/main
Updating 2423c22..a0758bf
Fast-forward
 .../model_prototyping/U-Net-FiLM-Training.ipynb    | 33 ++++++++++++++++++----
 1 file changed, 28 insertions(+), 5 deletions(-)


In [9]:
!pip install -r environment/requirements.txt

In [10]:
import sys
# Add src to path
sys.path.append(os.path.join(os.getcwd(), 'src'))

# Define Drive paths
DRIVE_DATA_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/data'
DRIVE_MODELS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/models'
DRIVE_RESULTS_PATH = '/content/drive/MyDrive/Thesis_DNN_Planner/results'

# Link Drive storage to local cloned directories
if not os.path.islink('data'):
    !ln -s {DRIVE_DATA_PATH} data
if not os.path.islink('models'):
    !ln -s {DRIVE_MODELS_PATH} models
if not os.path.islink('results'):
    !ln -s {DRIVE_RESULTS_PATH} results

print("✅ Environment setup complete.")

ln: failed to create symbolic link 'data/data': File exists
ln: failed to create symbolic link 'models/models': File exists
ln: failed to create symbolic link 'results/results': File exists
✅ Environment setup complete.


### Phase 2, Task 2.5: Full Pipeline Sanity Check

In [ ]:
import torch
import numpy as np
from pathlib import Path
from torch.utils.data import DataLoader
from dnn_guidance.data_loader import PathfindingDataset
from dnn_guidance.model import UNetFiLM
from dnn_guidance.loss import DiceFocalLoss

# Create a temporary minimal dataset
_temp_root = Path('tmp_sanity_data')
_samples = _temp_root/'samples'
_gt = _temp_root/'gt'
_samples.mkdir(parents=True, exist_ok=True)
_gt.mkdir(parents=True, exist_ok=True)

grid = np.zeros((200, 200), dtype=np.uint8)
grid[0, 0] = 8  # start
grid[-1, -1] = 9  # goal
np.savez(_samples/'sample0.npz', map=grid, clearance=2.0, step_size=8.0, config=np.array([]))
heatmap = np.zeros((200, 200), dtype=np.float32)
np.savez(_gt/'sample0.npz', heatmap=heatmap)

# Build dataset and dataloader
 dataset = PathfindingDataset(_samples, _gt)
loader = DataLoader(dataset, batch_size=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNetFiLM().to(device)
criterion = DiceFocalLoss()

(grid_batch, robot_batch), target_batch = next(iter(loader))
grid_batch = grid_batch.to(device)
robot_batch = robot_batch.to(device)
target_batch = target_batch.to(device)

logits = model(grid_batch, robot_batch)
print('Logits shape:', logits.shape)
loss = criterion(logits, target_batch)
print('Loss:', loss.item())

assert logits.shape == (1, 1, 200, 200)
assert loss.dim() == 0

loss.backward()
print('✅ Phase 2 Sanity Check Passed: Model, data pipeline, and loss function are fully integrated and functional.')